In [1]:
import numpy as np
import numba as nb

njit = nb.njit

In [2]:
W0 = 1            # For easier computations - results are linear anyway in W0
r = 0.005         # taux livret A
m = 0.0463        # From get-mean-std-CAC40-after-outbreak.py output
sigma = 0.0313    # From get-mean-std-CAC40-after-outbreak.py output
K = 5             # Number of years for the coupon
S = 100           # Number of Monte-Carlo simulations

In [3]:
def compute_mu(m, sigma, K, S):
    np.random.seed(0)
    return m + sigma*np.random.normal(size = (S, K))

@njit(nogil=True)
def njit_compute_mu(m, sigma, K, S):
    np.random.seed(0)
    epsilon = np.empty((S, K))
    for s in np.arange(S):
        for k in np.arange(K):
            epsilon[s, k] = np.random.normal()
    return m + sigma*epsilon

assert compute_mu(m, sigma, K, S).sum() == njit_compute_mu(m, sigma, K, S).sum()

In [4]:
@njit(nogil=True) # To increase performance
def compute_actualized_wealth(W0, r, mu, phi, c):
    W = np.zeros((mu.shape[0], mu.shape[1] + 1))
    W[:, 0] = W0
    for k in np.arange(mu.shape[1]):
        W[:, k + 1] = np.maximum(0, W[:, k] + (mu[:, k] - r)/(1 + r)*np.minimum(phi, W[:, k] - c) - c)
    return W

In [5]:
def compute_profitability_probability(W0, W):
    S = W.shape[0]
    K = W.shape[1] - 1
    inferior_boundary = W0*np.vstack([np.arange(K, 0, -1) for _ in np.arange(S)])/(K + 1)
    return (W[:, 1:] >= inferior_boundary).prod(axis = 1).mean()

@njit(nogil=True)
def njit_compute_profitability_probability(W0, W):
    S = W.shape[0]
    K = W.shape[1] - 1
    inferior_boundary = W0*np.arange(K, 0, -1)/(K + 1)
    p = 0
    for s in np.arange(S):
        product = 1
        for k in np.arange(K):
            product *= W[s, k + 1] >= inferior_boundary[k]
        p += product
    return p/S

phi = .5
c = 0.17
mu = compute_mu(m, sigma, 5, 5)
njit_mu = njit_compute_mu(m, sigma, 5, 5)
W = compute_actualized_wealth(W0, r, mu, phi, c)
njit_W = compute_actualized_wealth(W0, r, njit_mu, phi, c)

assert compute_profitability_probability(W0, W) == njit_compute_profitability_probability(W0, njit_W)

In [6]:
def compute__profitability_probability_map(W0, r, m, sigma, K, S, N_phi = 100, N_c = 100):
    p = np.inf*np.ones((N_c + 1, N_phi + 1))
    mu = compute_mu(m, sigma, K, S)
    W0_over_N_phi = W0/N_phi
    c_over_N_c = W0/N_c
    for n_c in np.arange(N_c + 1):
        c = c_over_N_c*n_c
        for n_phi in np.arange(N_phi + 1):
            phi = W0_over_N_phi*n_phi
            W = compute_actualized_wealth(W0, r, mu, phi, c)
            p[n_c, n_phi] = compute_profitability_probability(W0, W)
    return p

@njit(nogil=True)
def njit_compute__profitability_probability_map(W0, r, m, sigma, K, S, N_phi = 100, N_c = 100):
    p = np.inf*np.ones((N_c + 1, N_phi + 1))
    mu = njit_compute_mu(m, sigma, K, S)
    W0_over_N_phi = W0/N_phi
    c_over_N_c = W0/N_c
    for n_c in np.arange(N_c + 1):
        c = c_over_N_c*n_c
        for n_phi in np.arange(N_phi + 1):
            phi = W0_over_N_phi*n_phi
            W = compute_actualized_wealth(W0, r, mu, phi, c)
            p[n_c, n_phi] = njit_compute_profitability_probability(W0, W)
    return p

assert compute__profitability_probability_map(W0, r, m, sigma, 100, 100, N_phi = 100, N_c = 100).sum() == njit_compute__profitability_probability_map(W0, r, m, sigma, 100, 100, N_phi = 100, N_c = 100).sum()

In [7]:
%time mu = compute__profitability_probability_map(W0, r, m, sigma, K, S, N_phi = 1000, N_c = 1000)

Wall time: 4min 38s


In [8]:
%time mu = njit_compute__profitability_probability_map(W0, r, m, sigma, K, S, N_phi = 1000, N_c = 1000)

Wall time: 15.1 s
